In [ ]:
#@title Insatlling Pyorch

!pip install torch
!pip install torchvision

In [ ]:
#@title Import Dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import math

In [ ]:
#@title Define Hyperparameters

input_size = 3072 # img_size = (32,32) ---> three channels so 3*1024 in total\
num_hidden_layers = 8 #must be greater than 1
first_layer_size = 2000
min_layer_size = 250
hidden_sizes = [] # number of nodes at each hidden layer
for i in range(0, num_hidden_layers):
  hidden_sizes.append(first_layer_size - ( i*math.floor((first_layer_size - min_layer_size)/(num_hidden_layers - 1)) ))
print("Dimensions of each hidden layer: " + str(hidden_sizes))
num_classes = 100 # number of output classes discrete range [0,9]
num_epochs = 200 # number of times which the entire dataset is passed throughout the model
batch_size = 500 # the size of input data took for one iteration
lr = 1e-5 # size of step

Dimensions of each hidden layer: [2000, 1750, 1500, 1250, 1000, 750, 500, 250]
Dimensions of each hidden layer: [2000, 1750, 1500, 1250, 1000, 750, 500, 250]


In [ ]:
#@title Downloading CIFAR100 data

train_data = dsets.CIFAR100(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.CIFAR100(root = './data', train = False,
                       transform = transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#@title Loading the data

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)

In [ ]:
#@title Define model class

class Net(nn.Module): #inheriting from nn.Module
  def __init__(self, input_size, hidden_sizes, num_hidden_layers, num_classes):
    super(Net,self).__init__()
    self.networkLayers = []
    for i in range(0, num_hidden_layers):
      if (i == 0):
        self.networkLayers.append(nn.Linear(input_size, hidden_sizes[0]))
      else:
        self.networkLayers.append(nn.Linear(hidden_sizes[i - 1], hidden_sizes[i]))
    self.networkLayers.append(nn.Linear(hidden_sizes[i], num_classes)) #last layer
    self.relu = nn.ReLU()
    self.networkLayers = nn.ModuleList(self.networkLayers) #needed because layers are in a list

  def forward(self,x):
    out = x
    for i in range(0, num_hidden_layers):
      out = self.networkLayers[i](out)
      out = self.relu(out)
    return out

In [ ]:
#@title Build the model

net = Net(input_size, hidden_sizes, num_hidden_layers, num_classes)
if torch.cuda.is_available():
  net.cuda()

In [ ]:
#@title Define loss-function & optimizer

loss_function = nn.CrossEntropyLoss() #this is the logarithm of difference (log loss)
optimizer = torch.optim.Adam( net.parameters(), lr=lr)

In [ ]:
#@title Training the model

for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_gen):
    images = Variable(images.view(-1,3072)).cuda()
    labels = Variable(labels).cuda()

    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))

Epoch [1/200], Step [100/100], Loss: 5.2424
Epoch [2/200], Step [100/100], Loss: 5.1289
Epoch [3/200], Step [100/100], Loss: 5.1036
Epoch [4/200], Step [100/100], Loss: 5.1132
Epoch [5/200], Step [100/100], Loss: 5.0764
Epoch [6/200], Step [100/100], Loss: 5.1109
Epoch [7/200], Step [100/100], Loss: 5.1006
Epoch [8/200], Step [100/100], Loss: 4.9141
Epoch [9/200], Step [100/100], Loss: 4.8214
Epoch [10/200], Step [100/100], Loss: 4.7590
Epoch [11/200], Step [100/100], Loss: 4.8112
Epoch [12/200], Step [100/100], Loss: 4.7599
Epoch [13/200], Step [100/100], Loss: 4.6857
Epoch [14/200], Step [100/100], Loss: 4.7082
Epoch [15/200], Step [100/100], Loss: 4.7478
Epoch [16/200], Step [100/100], Loss: 4.6664
Epoch [17/200], Step [100/100], Loss: 4.6929
Epoch [18/200], Step [100/100], Loss: 4.7184
Epoch [19/200], Step [100/100], Loss: 4.6178
Epoch [20/200], Step [100/100], Loss: 4.6676
Epoch [21/200], Step [100/100], Loss: 4.7981
Epoch [22/200], Step [100/100], Loss: 4.7251
Epoch [23/200], Ste

In [ ]:
#@title Evaluating the accuracy of the model

correct = 0
total = 0
for images,labels in test_gen:
  images = Variable(images.view(-1,3072)).cuda()
  labels = labels.cuda()

  output = net(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

Accuracy of the model: 16.000 %
Accuracy of the model: 16.000 %
